# Import WOS & Scopus vers Infoscience : documentation des classes & fonctions + exemples

In [1]:
import sys
import os
import pandas as pd
import json

sys.path.append(os.path.abspath(".."))

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

***

## Exemples d'utilisation des clients : moissonnage des sources

In [2]:
from clients.wos_client_v2 import WosClient
from clients.scopus_client import ScopusClient

In [3]:
wos_epfl_query = "OG=(Ecole Polytechnique Federale de Lausanne)"
scopus_epfl_query = "AF-ID(60028186) AND (ORIG-LOAD-DATE AFT 20240722) AND (ORIG-LOAD-DATE BEF 20240831)" # pour Scopus le range de dates fait partie de la query
createdTimeSpan = "2024-01-01+2024-01-03" # Pour le WoS le range de date est un paramètre supplémentaire

### Nombre total de résultats

In [ ]:
## WoS
WosClient.count_results(usrQuery=wos_epfl_query)
WosClient.count_results(usrQuery=wos_epfl_query, createdTimeSpan=createdTimeSpan)

In [ ]:
## Scopus
ScopusClient.count_results(query=scopus_epfl_query)

### Récupération des IDs Scopus et WoS

Par défaut count = 10 

In [ ]:
## WoS
WosClient.fetch_ids(usrQuery=wos_epfl_query, count=2,createdTimeSpan=createdTimeSpan)

In [ ]:
## Scopus
ScopusClient.fetch_ids(query=scopus_epfl_query)

In [ ]:
## Loop
total = ScopusClient.count_results(query=scopus_epfl_query)
count = 5
ids = []
for i in range(1, int(total), int(count)):
    ids.extend(ScopusClient.fetch_ids(query = scopus_epfl_query, count = count, start =i))

### Extraction des métadonnées

4 formats de sortie possibles :

- "**digest**" (défault) :retourne les métadonnées
  - source
  - internal_id
  - doi (**Important : le DOI est convert en lowercase**)
  - title
  - doctype
  - pubyear
- "**digest-ifs3**" : retourne les métadonnées du format **digest** plus
  - ifs3_doctype (nom de la collection Infoscience)
  - ifs3_collection_id (uuid de la collection Infoscience)
- "**ifs3**" : retourne les métadonnnées du format **digest-ifs3** plus
  - authors : liste d'objets auteur comprenant les métadonnées :
    - author (nom de l'auteur)
    - internal_author_id
    - orcid_id,
    - organizations
    - sub_organizations
- "**scopus**" ou "**wos**" : formats natifs du WoS ou Scopus


Par défault le format de sortie est "digest"

In [ ]:
# WoS format digest
WosClient.fetch_records(usrQuery=wos_epfl_query,count=2,createdTimeSpan=createdTimeSpan, format="ifs3")

In [ ]:
# Scopus format ifs3
ScopusClient.fetch_records(format="ifs3",query=scopus_epfl_query,count=2)

In [ ]:
# Loop
total = ScopusClient.count_results(query=scopus_epfl_query)
count = 50
recs = []
for i in range(1, int(total), int(count)):
    recs.extend(ScopusClient.fetch_records(query = scopus_epfl_query, count = count, start =i))

## Exemples d'utilisation des clients : retrieval

### Retrieval api.epfl.ch

In [5]:
from clients.api_epfl_client import ApiEpflClient

#### Endpoints persons

In [ ]:
# param fistname default None
# param lastname default None
# param use_firstname_lastname default False (: ne pas utiliser la recherche sur firstname-lastname)
# param format default "sciper" (: ne retourne que le sciper Id si trouvé. Autres formats : "digest" (sciper Id + units Ids) et "epfl" (all))

ApiEpflClient.query_person("a bay", format="digest")

In [ ]:
ApiEpflClient.query_person(
    "O Schneider",
    firstname="schneider",
    lastname="o",
    format="sciper",
    use_firstname_lastname=True,
)

In [ ]:
ApiEpflClient.query_person("bay a", format="epfl")

#### Endpoint accreds

Les données des unités récupérées via accred sont enrichies avec la métadonnée du type de l'unité (obtenue en requêtant l'endpoint api.epfl.ch/units)

In [ ]:
#param format default "mainUnit" (: ne renvoie que la 1ère paire unit_id/unit_name retournée par accred. Autres formats : "digest" (toutes les paires unit_id/unit_name), "epfl" (all)) 
ApiEpflClient.fetch_accred_by_unique_id("105958")

#### Endpoint units

In [ ]:
#param format default "digest" (: ne renvoie que le type de l'unité retourné par unit. Autres formats : "epfl" (all record)) 
ApiEpflClient.fetch_unit_by_unique_id("10913")

In [ ]:
ApiEpflClient.fetch_unit_by_unique_id("10913", format="epfl")

### Retrieval Unpaywall 

Pour un DOI :

2 formats de sortie possibles : 

- **"oa"** : retourne les métadonnées
  - is_oa
  - oa_status
- **"oa-locations" (default)** : retourne les métadonnées du format **oa** plus :
  - pdf_urls : si is_oa est True + si oa_status est gold ou hybrid + si la version est "publishedVersion" avec license cc-by dans chaque oa_locations
     

**Todo : requête sur titre-auteur quand la publi n'a pas de doi**

In [1]:
from clients.unpaywall_client import UnpaywallClient

In [ ]:
#param format default "oa-locations" (: renvoie is_oa, oa_status et pdf_urls. Autres formats : "oa" (seulement is_oa et oa_status) et "upw" (all)) 
UnpaywallClient.fetch_by_doi("10.1016/j.apenergy.2024.124273")

In [ ]:
UnpaywallClient.fetch_by_doi("10.1016/j.apenergy.2024.124273", format="oa")

## Autres clients de retrieval (inutilisés)

### Retrieval service Istex orcidDisambiguation

In [4]:
from clients.services_istex_client import ServicesIstexClient

In [ ]:
# Retourne l'Orcid Id
ServicesIstexClient.get_orcid_id(firstname="M.G.", lastname="Preti")

### Retrieval API Orcid

In [ ]:
from clients.orcid_client import OrcidClient

In [ ]:
# le format par défaut est "digest"
OrcidClient.fetch_record_by_unique_id("0000-0001-9511-1958") # same as OrcidClient.fetch_record_by_unique_id("0000-0001-9511-1958", format="digest")

In [ ]:
OrcidClient.fetch_record_by_unique_id("0000-0001-9511-1958", format="orcid") # retourne le record Orcid complet

In [ ]:
ApiEpflClient.fetch_accred_by_unique_id("105958", format="digest")

***

## Exemples d'utilisation des harvesters

Les harvesters permettent d'unifier le processus de moissonnage multi-sources et de produire les dataframes des publications

In [ ]:
from data_pipeline.harvester import WosHarvester, ScopusHarvester

In [ ]:
default_queries = {
        "wos": "OG=(Ecole Polytechnique Federale de Lausanne)",
        "scopus": "AF-ID(60028186) OR AF-ID(60210159) OR AF-ID(60070536) OR AF-ID(60204330) OR AF-ID(60070531) OR AF-ID(60070534) OR AF-ID(60070538) OR AF-ID(60014951) OR AF-ID(60070529) OR AF-ID(60070532) OR AF-ID(60070535) OR AF-ID(60122563) OR AF-ID(60210160) OR AF-ID(60204331)",
        "openalex": "OPENALEX_QUERY_HERE",  # Placeholder for OpenAlex query in the future ?
        "zenodo": "ZENODO_QUERY_HERE"      # Placeholder for Zenodo query in teh future ?
    }
start_date = "2024-07-01"
end_date = "2024-07-10"

Par défaut le format de sortie est "ifs3"

In [ ]:
# Dataframe des publications WoS
wos_harvester = WosHarvester(start_date, end_date, default_queries["wos"])
wos_publications = wos_harvester.harvest()

In [ ]:
# Dataframe des publications Scopus
## format par défaut : ifs3
scopus_harvester = ScopusHarvester(start_date, end_date, default_queries["scopus"])
scopus_publications = scopus_harvester.harvest()

***

## Dédoublonnage

In [ ]:
from data_pipeline.deduplicator import DataFrameProcessor

In [ ]:
# Merge 
deduplicator = DataFrameProcessor(wos_publications, scopus_publications)
# Deduplicate the publications : first deduplicate operation between the sources
deduplicated_sources_df = deduplicator.deduplicate_dataframes()
# and second operation : filter by removing founded duplicates in Infoscience
df_final,df_unloaded = deduplicator.deduplicate_infoscience(deduplicated_sources_df)
# Generate main dataframes
df_metadata, df_authors = deduplicator.generate_main_dataframes(df_final)

In [ ]:
df_metadata = pd.read_csv("harvested-data/2024_09_15/ResearchOutput.csv", encoding="utf-8")
df_metadata

***

## Reconciliation avec les autorités Infoscience

In [ ]:
from clients.dspace_client_wrapper import DSpaceClientWrapper

dspace_wrapper = DSpaceClientWrapper()

In [ ]:
# query = "0000-0002-8826-3870"
query = "blanc f"
response = dspace_wrapper._search_authority(filter_text=query)
print(response)
sciper_id = dspace_wrapper.get_sciper_from_authority(response)

print(f"sciper : {sciper_id}")

***

## Consolidation métadonnées auteurs avec Infoscience et api.epfl.ch

In [ ]:
from data_pipeline.enricher import AuthorProcessor

In [ ]:
df = pd.read_csv(
    "./harvested-data/2024_11_07-17_04_02/AddressesAndNames.csv", encoding="utf-8"
)
df.shape

### Etape 1 : Détection des auteurs affiliés à l'EPFL + nettoyage des noms auteurs

In [5]:
# epfl_affiliation = True/False
processor = AuthorProcessor(df)
df_aff = processor.process(return_df=True)

In [ ]:
# filtre sur les auteurs EPFL
processor = AuthorProcessor(df_aff)
df_epfl = processor.filter_epfl_authors(return_df=True)
df_epfl.shape

In [ ]:
# nouvelles colonnes "author_cleaned", "nameparse_firstanme" et "nameparse_lastname")
processor = AuthorProcessor(df_epfl)
df_cleaned_names = processor.clean_authors().nameparse_authors(return_df=True)
df_cleaned_names.shape

In [ ]:
df_cleaned_names

### Etape 2 : Récupération des infos des unités via api.epfl.ch

Pour chaque auteur :

- requêtage api.epfl.ch/persons sur le triplet author_name|nameparse_firstname|namepars_lastname pour obtenir un sciper_id
- requêtage api.pefl.ch/accreds sur le sciper_id pour obtenir l'unité principale
  - on récupère d'abord toutes les unités de l'auteur dans accred en respectant l'ordre
  - on enrichit chaque unité en requêtant api.epfl.ch/units sur le unit_id pour obtenir le unittype
  - on boucle sur la liste des unités jusqu'à trouver une unité de type 'Laboratoire'
    - si on trouve une telle unité on retourne la paire unit-id|unit_name correspondante
    - si on ne trouve pas d'uniét de type 'Laboratoire' on retourne al paire unit_id|unit_name de la 1ère occurence dans accred

In [ ]:
processor = AuthorProcessor(df_cleaned_names)
df_api_epfl_infos = processor.api_epfl_reconciliation(return_df=True)

In [ ]:
df_api_epfl_infos

### Etape 3 : recherche dans les entités Dspace (ajout des uuid des entités auteurs si trouvés dans DSpace)

In [ ]:
Pour chaque auteur : 

- requête du client DSpace sur le sciper_id s'il a été récupéré
- sinon requête du client Dspace sur le author_name

In [ ]:
processor = AuthorProcessor(df_api_epfl_infos)
df_dspace_infos = processor.generate_dspace_uuid(return_df=True)

In [ ]:
df_dspace_infos

### All in one :  pipeline

In [ ]:
processor = AuthorProcessor(df)
df_api_epfl_infos = (processor
                     .process()
                     .filter_epfl_authors()
                     .clean_authors()
                     .nameparse_authors()
                     .api_epfl_reconciliation()
                     .generate_dspace_uuid(return_df=True)
                    ) 

### Etapes complémentaires (inutilisées)

In [ ]:
processor = AuthorProcessor(df_cleaned_names[0:20])
df_api_epfl_infos = processor.services_istex_orcid_reconciliation().orcid_data_reconciliation(return_df=True)

In [ ]:
processor = Processor(df)
df_api_epfl_infos = (processor
                     .process()
                     .filter_epfl_authors()
                     .clean_authors()
                     .nameparse_authors()
                     .api_epfl_reconciliation()
                     .services_istex_orcid_reconciliation()
                     .orcid_data_reconciliation()
                     .api_epfl_reconciliation()
                     .generate_dspace_uuid(return_df=True)
                    )                    

***

## Consolidation métadonnées publications avec le client Unpaywall

Pour chaque publi :

- requêtage unpaywall sur le DOI
  - on récupère d'abord les principaux attributs OA : is_oa et oa_status
  - Si is_oa est True :
    - on boucle sur la liste des unités jusqu'à trouver une unité de type 'Laboratoire'
    - si on trouve une telle unité on retourne la paire unit-id|unit_name correspondante
    - si on ne trouve pas d'uniét de type 'Laboratoire' on retourne al paire unit_id|unit_name de la 1ère occurence dans accred

In [3]:
from data_pipeline.enricher import PublicationProcessor

In [ ]:
df_metadata = pd.read_csv("harvested-data/2024_09_15/ResearchOutput.csv", encoding="utf-8")
df_metadata.shape

In [ ]:
processor = PublicationProcessor(df_metadata)
df_upw_metadata = processor.process(return_df=True)

In [ ]:
df_upw_metadata

## Loader

In [ ]:
from data_pipeline.loader import Loader

In [ ]:
# Create publications in Dspace
Loader.create_complete_publication(df_metadata)
#  WIP Create or update person entities in Dspace
Loader.manage_person(df_api_epfl_info)